In [409]:
from pyscf import gto, scf, ao2mo, fci
from GeneralisedWick import *
from scipy.optimize import fsolve

In [410]:
def commutator(operator1, operator2):
    return operator1 * operator2 + (-1) * operator2 * operator1

In [411]:
def BCHSimilarityTransform(H, T, order):
    result = H
    for k in range(order):
        nestedCommutator = H
        for i in range(k + 1):
            nestedCommutator = commutator(nestedCommutator, T)
        result += (1 / factorial(k + 1)) * nestedCommutator
    return result

In [412]:
def projectionManifold(excitationLevel):
    return Tensor('\Phi', ['h'] * excitationLevel, ['p'] * excitationLevel)

In [413]:
fockTensor = Tensor("f", ['g'], ['g'])
h1Tensor = Tensor("h", ['g'], ['g'])
h2Tensor = Tensor("v", ['g', 'g'], ['g', 'g'])
t2Tensor = Tensor("{t_{2}}", ['p', 'p'], ['h', 'h'])

In [414]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
#mol.atom = 'Be 0 0 0'
#mol.basis = 'sto-3g'
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = '6-31g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

cisolver = fci.FCI(mol, mf.mo_coeff)

Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1] * Nocc + [0] * (Norbs - Nocc)

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff, compact=False)

In [415]:
from pyscf import cc, ccn

In [416]:
trueCCD = cc.CCSD(mf).kernel()

In [417]:
trueCCD

(-0.04675404036274192,
 (array([[0.        , 0.        , 0.        , 0.00034378, 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.01691119, 0.        ,
          0.        , 0.        ]]),
  array([[0.        , 0.        , 0.        , 0.00034378, 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.01691119, 0.        ,
          0.        , 0.        ]])),
 (array([[[[ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,

In [418]:
trueAmplitudeArray = trueCCD[2][1].swapaxes(0,2).swapaxes(1,3)

In [419]:
trueAmplitudeArray

array([[[[ 7.19737878e-05, -1.22968729e-03],
         [-1.22968729e-03, -1.71537433e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-8.29953157e-04, -1.24378783e-03],
         [-2.37889655e-03, -5.73075335e-02]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]]],


       [[[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 7.19737878e-05, -1.22968729e-03],
         [-1.22968729e-03, -1.71537433e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.000

In [420]:
#t2Tensor.array

In [421]:
def get2bodyHamiltonianArray(mf_):
    eri = ao2mo.kernel(mf_.mol, mf_.mo_coeff)
    Norbs_ = mf_.mol.nao
    hamiltonian2BodyArray = np.zeros((Norbs_,Norbs_,Norbs_,Norbs_))
    for p in range(Norbs_):
        for q in range(Norbs_):
            for r in range(p + 1):
                for s in range(q + 1):
                    x = int(p + Norbs_ * r - 0.5 * r * (r + 1))
                    y = int(q + Norbs_ * s - 0.5 * s * (s + 1))
                    if p == r and q == s:
                        hamiltonian2BodyArray[p,q,r,s] += eri[x, y]
                    else:
                        hamiltonian2BodyArray[p,q,r,s] += eri[x, y]
                        hamiltonian2BodyArray[r,s,p,q] += np.conjugate(eri[x, y])
    return hamiltonian2BodyArray

In [422]:
h1Tensor.array = h1
h2Tensor.array = eri.reshape((Norbs, Norbs, Norbs, Norbs)).swapaxes(2,3).swapaxes(1,2)
t2Tensor.getShape(vacuum)

In [423]:
#t2Tensor.array[0, 0, 0, 0] = -0.275333314977893 * 0.5
t2Tensor.array = np.zeros(t2Tensor.array.shape)
#t2Tensor.array = trueAmplitudeArray

In [424]:
h1

array([[-7.94050428,  0.20663739,  0.        ,  0.        ,  0.        ,
         0.17965963,  0.        ,  0.        ,  0.        ],
       [ 0.20663739, -1.61807386,  0.        ,  0.        ,  0.        ,
        -0.43217769,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.19628378,  0.        ,  0.        ,
         0.        , -0.48516619,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -1.19628378,  0.        ,
         0.        ,  0.        ,  0.        , -0.48516619],
       [ 0.        ,  0.        ,  0.        ,  0.        , -1.19628378,
         0.        ,  0.        , -0.48516619,  0.        ],
       [ 0.17965963, -0.43217769,  0.        ,  0.        ,  0.        ,
        -0.95456897,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.48516619,  0.        ,  0.        ,
         0.        , -1.3410308 ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0

In [425]:
mf.mo_energy

NPArrayWithTag([-4.7068905 , -0.30129539,  0.08243534,  0.08243534,
                 0.08243534,  0.43975431,  0.46493101,  0.46493101,
                 0.46493101])

In [426]:
eri

array([[ 2.27130649, -0.19996916,  0.        , ...,  0.        ,
         0.        ,  0.58302395],
       [-0.19996916,  0.0283882 ,  0.        , ...,  0.        ,
         0.        , -0.0096241 ],
       [ 0.        ,  0.        ,  0.00330086, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.01877945,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.01877945,  0.        ],
       [ 0.58302395, -0.0096241 ,  0.        , ...,  0.        ,
         0.        ,  0.39402566]])

In [427]:
h2Tensor.array

array([[[[ 2.27130649e+00, -1.99969155e-01,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [-1.99969155e-01,  2.83882044e-02,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  3.30086167e-03, ...,
           9.35905544e-03,  0.00000000e+00,  0.00000000e+00],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  9.35905544e-03, ...,
           2.69900982e-02,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  2.69900982e-02,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  2.69900982e-02]],

        [[-1.99969155e-01,  4.95347745e-01,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 2.83882044e-02, -6.66823217e-03,  0.00000000e+00, ...,
           0.00000000e

In [428]:
#fock = mf.get_fock(h1e=h1, vhf=eri)

In [429]:
#print(fock)
#print(np.linalg.eig(fock))

In [430]:
eri.reshape((Norbs, Norbs, Norbs, Norbs)).swapaxes(2,3).swapaxes(1,2)

array([[[[ 2.27130649e+00, -1.99969155e-01,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [-1.99969155e-01,  2.83882044e-02,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  3.30086167e-03, ...,
           9.35905544e-03,  0.00000000e+00,  0.00000000e+00],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  9.35905544e-03, ...,
           2.69900982e-02,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  2.69900982e-02,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  2.69900982e-02]],

        [[-1.99969155e-01,  4.95347745e-01,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 2.83882044e-02, -6.66823217e-03,  0.00000000e+00, ...,
           0.00000000e

In [431]:
fock = h1
for p in range(Norbs):
    for q in range(Norbs):
        fock[p,q] += sum([2 * h2Tensor.array[p,i,q,i] - h2Tensor.array[p,i,i,q] for i in range(Nocc)])

In [432]:
fock

array([[-4.70689051e+00,  5.31283906e-11,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  3.21496441e-11,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 5.31285016e-11, -3.01295386e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  8.48783932e-10,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  8.24353414e-02,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -6.09065243e-10,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         8.24353414e-02,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -6.09065409e-10],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  8.24353414e-02,  0.00000000e+00,
         0.00000000e+00, -6.09065409e-10,  0.00000000e+00],
       [ 3.21496163e-11,  8.48783821e-10,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  4.397543

In [433]:
fockTensor.array = fock

In [434]:
np.linalg.eig(fock)

(array([-4.70689051, -0.30129539,  0.43975431,  0.08243534,  0.46493101,
         0.08243534,  0.46493101,  0.46493101,  0.08243534]),
 array([[-1.00000000e+00,  1.20592994e-11,  6.24671903e-12,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.20593246e-11,  1.00000000e+00,  1.14538049e-09,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00, -1.00000000e+00,
          1.59234547e-09,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -1.00000000e+00,  1.59234591e-09,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e

In [435]:
mf.mo_energy

NPArrayWithTag([-4.7068905 , -0.30129539,  0.08243534,  0.08243534,
                 0.08243534,  0.43975431,  0.46493101,  0.46493101,
                 0.46493101])

Hard code spin-free equations from Bartlett & Shavitt Chapter 12

In [436]:
fockDiagrams = fockTensor.getAllDiagrams(vacuum)
h2Diagrams = h2Tensor.getAllDiagrams(vacuum)

In [437]:
print(*fockDiagrams)

f_{h}^{h} f_{h}^{p} f_{p}^{h} f_{p}^{p}


In [438]:
print([fockDiagram.array.shape for fockDiagram in fockDiagrams])

[(2, 2), (2, 7), (7, 2), (7, 7)]


In [439]:
print(*h2Diagrams)

v_{hh}^{hh} v_{hh}^{hp} v_{hh}^{ph} v_{hh}^{pp} v_{hp}^{hh} v_{hp}^{hp} v_{hp}^{ph} v_{hp}^{pp} v_{ph}^{hh} v_{ph}^{hp} v_{ph}^{ph} v_{ph}^{pp} v_{pp}^{hh} v_{pp}^{hp} v_{pp}^{ph} v_{pp}^{pp}


In [440]:
print([h2Diagram.array.shape for h2Diagram in h2Diagrams])

[(2, 2, 2, 2), (2, 2, 2, 7), (2, 2, 7, 2), (2, 2, 7, 7), (2, 7, 2, 2), (2, 7, 2, 7), (2, 7, 7, 2), (2, 7, 7, 7), (7, 2, 2, 2), (7, 2, 2, 7), (7, 2, 7, 2), (7, 2, 7, 7), (7, 7, 2, 2), (7, 7, 2, 7), (7, 7, 7, 2), (7, 7, 7, 7)]


In [441]:
I = Index("I", True)
J = Index("J", True)
K = Index("K", True)
L = Index("L", True)
A = Index("A", False)
B = Index("B", False)
C = Index("C", False)
D = Index("D", False)

In [442]:
D1 = TensorSum([TensorProduct([h2Diagrams[12]], vertexList=[Vertex(h2Diagrams[12], [A,B], [I,J])])])

In [443]:
D2a = TensorSum([TensorProduct([fockDiagrams[3], t2Tensor], vertexList=[Vertex(fockDiagrams[3], [B], [C]), Vertex(t2Tensor, [A,C], [I,J])]), TensorProduct([fockDiagrams[3], t2Tensor], vertexList=[Vertex(fockDiagrams[3], [A], [C]), Vertex(t2Tensor, [B,C], [J,I])])])

In [444]:
D2b = TensorSum([TensorProduct([fockDiagrams[0], t2Tensor], vertexList=[Vertex(fockDiagrams[0], [K], [J]), Vertex(t2Tensor, [A,B], [I,K])]), TensorProduct([fockDiagrams[0], t2Tensor], vertexList=[Vertex(fockDiagrams[0], [K], [I]), Vertex(t2Tensor, [B,A], [J,K])])])

In [445]:
D2c = TensorSum([TensorProduct([h2Diagrams[15], t2Tensor], vertexList=[Vertex(h2Diagrams[15], [A,B], [C,D]), Vertex(t2Tensor, [C,D], [I,J])]), TensorProduct([h2Diagrams[15], t2Tensor], vertexList=[Vertex(h2Diagrams[15], [B,A], [C,D]), Vertex(t2Tensor, [C,D], [J,I])])])

In [446]:
D2d = TensorSum([TensorProduct([h2Diagrams[0], t2Tensor], vertexList=[Vertex(h2Diagrams[0], [K,L], [I,J]), Vertex(t2Tensor, [A,B], [K,L])]), TensorProduct([h2Diagrams[0], t2Tensor], vertexList=[Vertex(h2Diagrams[0], [K,L], [J,I]), Vertex(t2Tensor, [B,A], [K,L])])])

In [447]:
D2e = TensorSum([TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,B], [C,J]), Vertex(t2Tensor, [A,C], [I,K])]), TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,A], [C,I]), Vertex(t2Tensor, [B,C], [J,K])])])

In [448]:
D2ex1 = TensorSum([TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,B], [C,J]), Vertex(t2Tensor, [A,C], [K,I])]), TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,A], [C,I]), Vertex(t2Tensor, [B,C], [K,J])])])

In [449]:
D2ex2 = TensorSum([TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,B], [I,C]), Vertex(t2Tensor, [A,C], [K,J])]), TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,A], [J,C]), Vertex(t2Tensor, [B,C], [K,I])])])

In [450]:
D2ex3 = TensorSum([TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,B], [J,C]), Vertex(t2Tensor, [A,C], [I,K])]), TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,A], [I,C]), Vertex(t2Tensor, [B,C], [J,K])])])

In [451]:
D3a = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [I,J]), Vertex(t2Tensor, [A,B], [K,L])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [J,I]), Vertex(t2Tensor, [B,A], [K,L])])])

In [452]:
D3b = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,C], [I,K]), Vertex(t2Tensor, [D,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,C], [J,K]), Vertex(t2Tensor, [D,A], [L,I])])])

In [453]:
D3bx1 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,C], [I,K]), Vertex(t2Tensor, [D,B], [J,L])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,C], [J,K]), Vertex(t2Tensor, [D,A], [I,L])])])

In [454]:
D3bx2 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,A], [I,K]), Vertex(t2Tensor, [B,D], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,B], [J,K]), Vertex(t2Tensor, [A,D], [L,I])])])

In [455]:
D3bx3 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,D], [I,K]), Vertex(t2Tensor, [C,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,D], [J,K]), Vertex(t2Tensor, [C,A], [L,I])])])

In [456]:
D3bx4 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,D], [K,I]), Vertex(t2Tensor, [C,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,D], [K,J]), Vertex(t2Tensor, [C,A], [L,I])])])

In [457]:
D3bx5 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,B], [I,L]), Vertex(t2Tensor, [A,D], [K,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,A], [J,L]), Vertex(t2Tensor, [B,D], [K,I])])])

In [458]:
D3c = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [K,I]), Vertex(t2Tensor, [A,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [K,J]), Vertex(t2Tensor, [B,A], [L,I])])])

In [459]:
D3cx = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [I,K]), Vertex(t2Tensor, [A,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [J,K]), Vertex(t2Tensor, [B,A], [L,I])])])

In [460]:
D3d = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,A], [K,L]), Vertex(t2Tensor, [D,B], [I,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,B], [K,L]), Vertex(t2Tensor, [D,A], [J,I])])])

In [461]:
D3dx = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,C], [K,L]), Vertex(t2Tensor, [D,B], [I,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,C], [K,L]), Vertex(t2Tensor, [D,A], [J,I])])])

In [462]:
amplitudeDiagrams = [D1, D2a, D2b, D2c, D2d, D2e, D2ex1, D2ex2, D2ex3, D3a, D3b, D3bx1, D3bx2, D3bx3, D3bx4, D3bx5, D3c, D3cx, D3d, D3dx]
amplitudeFactors = [1., 1., -1., 0.5, 0.5, 2., -1., -1., -1., 0.5, 2., -2., 0.5, -1., 1., 0.5, -2., 1., -2., 1.]

In [463]:
print(D2a)

1.0 * f_{B}^{C}{t_{2}}_{AC}^{IJ}
 + 1.0 * f_{A}^{C}{t_{2}}_{BC}^{JI}


In [464]:
D2a.summandList[0].vertexList[1].tensor.array.shape

(7, 7, 2, 2)

In [465]:
contractTensorSum(TensorSum([D2a.summandList[0]])).shape

(7, 7, 2, 2)

In [466]:
def testContractedArray(tensorProduct_, targetLowerIndexList=[A,B], targetUpperIndexList=[I,J]):
    lowerIndexList = list(itertools.chain.from_iterable([vertex.lowerIndices for vertex in tensorProduct_.vertexList]))
    upperIndexList = list(itertools.chain.from_iterable([vertex.upperIndices for vertex in tensorProduct_.vertexList]))
    lowerIndexLetters = string.ascii_lowercase[:len(lowerIndexList)]
    upperIndexLetters = ''
    freeLowerIndexMask = np.ones(len(lowerIndexList))
    freeUpperIndexMask = np.ones(len(upperIndexList))
    nFreeUpperIndices = 0
    for uI, upperIndex in enumerate(upperIndexList):
        free = True
        for lI, lowerIndex in enumerate(lowerIndexList):
            if upperIndex == lowerIndex:
                #print(upperIndex, lowerIndex)
                upperIndexLetters += lowerIndexLetters[lI]
                freeLowerIndexMask[lI] = 0
                freeUpperIndexMask[uI] = 0
                free = False
        if free:
            upperIndexLetters += string.ascii_lowercase[len(lowerIndexList) + nFreeUpperIndices]
            nFreeUpperIndices += 1
    newLowerIndexList = [lowerIndex for lI, lowerIndex in enumerate(lowerIndexList) if freeLowerIndexMask[lI]]
    newUpperIndexList = [upperIndex for uI, upperIndex in enumerate(upperIndexList) if freeUpperIndexMask[uI]]
    summandZero = False
    if targetLowerIndexList == None and targetUpperIndexList == None:
        targetLowerIndexList = newLowerIndexList
        targetUpperIndexList = newUpperIndexList
        summandZero = True
    freeLowerIndexLetters = "".join([lowerIndexLetters[lowerIndexList.index(lowerIndex)] for lowerIndex in targetLowerIndexList])
    freeUpperIndexLetters = "".join([upperIndexLetters[upperIndexList.index(upperIndex)] for upperIndex in targetUpperIndexList])
    einsumSubstrings = []
    start = 0
    for vertex in tensorProduct_.vertexList:
        end = start + vertex.excitationRank
        einsumSubstring = lowerIndexLetters[start:end] + upperIndexLetters[start:end]
        einsumSubstrings.append(einsumSubstring)
        start = end
    einsumString = ",".join(einsumSubstrings)
    einsumString += '->' + freeLowerIndexLetters + freeUpperIndexLetters
    print(*lowerIndexList, *upperIndexList)
    print(*lowerIndexLetters, *upperIndexLetters)
    print(*newLowerIndexList, *newUpperIndexList)
    print(*freeLowerIndexLetters, *freeUpperIndexLetters)
    print(einsumString)
    contribution = tensorProduct_.prefactor * np.einsum(einsumString, *[vertex.tensor.array for vertex in tensorProduct_.vertexList])
    #print(*newLowerIndexList, *newUpperIndexList)
    #if summandZero:
    #    return contribution, newLowerIndexList, newUpperIndexList
    return contribution

In [467]:
t2Tensor.array.shape

(7, 7, 2, 2)

In [468]:
tCA = testContractedArray(D2a.summandList[0])
print(D2a.summandList[1])
print(tCA)
print(tCA.shape)
#print(*tCA[1])
#print(*tCA[2])

B A C C I J
a b c c d e
B A I J
b a d e
ac,bcde->bade
1.0 * f_{A}^{C}{t_{2}}_{BC}^{JI}
[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [

In [469]:
amplitudes = TensorSum([])
for d, diagram in enumerate(amplitudeDiagrams):
    for summand in diagram.summandList:
        amplitudes.summandList.append(copy(summand))
        amplitudes.summandList[-1].prefactor *= amplitudeFactors[d]
    print(diagram)
    print(contractTensorSum(diagram))
#    print(amplitudeFactors[d] * contractTensorSum(diagram))

1.0 * v_{AB}^{IJ}
[[[[ 0.00330086  0.00659818]
   [ 0.00659818  0.063035  ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00935906  0.00673648]
   [ 0.01561124  0.02848843]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]]


 [[[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00330086  0.00659818]
   [ 0.00659818  0.063035  ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00935906  0.00673648]
   [ 0.01561124  0.02848843]]]


 [[[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00330086  0

In [470]:
#t2Tensor.array[0, 0, 0, 0] = -0.38

In [471]:
h2Tensor.array = get2bodyHamiltonianArray(mf)

In [472]:
residualTerms = [amplitudeFactors[d] * contractTensorSum(diagram) for d, diagram in enumerate(amplitudeDiagrams)]

In [473]:
sum([testContractedArray(term) for term in amplitudes.summandList])

A B I J
a b c d
A B I J
a b c d
abcd->abcd
B A C C I J
a b c c d e
B A I J
b a d e
ac,bcde->bade
A B C C J I
a b c c d e
A B J I
a b e d
ac,bcde->abed
K A B J I K
a b c d e a
A B J I
b c e d
ad,bcea->bced
K B A I J K
a b c d e a
B A I J
c b d e
ad,bcea->cbde
A B C D C D I J
a b c d c d e f
A B I J
a b e f
abcd,cdef->abef
B A C D C D J I
a b c d c d e f
B A J I
b a f e
abcd,cdef->bafe
K L A B I J K L
a b c d e f a b
A B I J
c d e f
abef,cdab->cdef
K L B A J I K L
a b c d e f a b
B A J I
d c f e
abef,cdab->dcfe
K B A C C J I K
a b c d d e f a
B A J I
c b f e
abde,cdfa->cbfe
K A B C C I J K
a b c d d e f a
A B I J
b c e f
abde,cdfa->bcef
K B A C C J K I
a b c d d e a f
B A J I
c b f e
abde,cdaf->cbfe
K A B C C I K J
a b c d d e a f
A B I J
b c e f
abde,cdaf->bcef
K B A C I C K J
a b c d e d a f
B A I J
c b e f
abed,cdaf->cbef
K A B C J C K I
a b c d e d a f
A B J I
b c f e
abed,cdaf->bcfe
K B A C J C I K
a b c d e d f a
B A J I
c b f e
abed,cdfa->cbfe
K A B C I C J K
a b c d e d f a
A B I

array([[[[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00935906,  0.00673648],
         [ 0.01561124,  0.02848843]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [474]:
contractTensorSum(amplitudes)

array([[[[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00935906,  0.00673648],
         [ 0.01561124,  0.02848843]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [475]:
residualTerms

[array([[[[ 0.00330086,  0.00659818],
          [ 0.00659818,  0.063035  ]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.00935906,  0.00673648],
          [ 0.01561124,  0.02848843]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]]],
 
 
        [[[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.00330086,  0.00659818],
          [ 0.00659818,  0.063035  ]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.        ,  0.        ],
          [ 0.        ,  0.        ]],
 
         [[ 0.        ,  0.        ],
          [

In [476]:
constant = sum(residualTerms[:1])
linearFock = sum(residualTerms[1:3])
linearFluctuation = sum(residualTerms[3:9])
quadratic = sum(residualTerms[9:])

In [477]:
supposedLinearFluctuation = residualTerms[3] - residualTerms[4]
print(supposedLinearFluctuation)

[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0.

In [478]:
print(constant)
print(linearFock)
print(linearFluctuation)
print(quadratic)

[[[[ 0.00330086  0.00659818]
   [ 0.00659818  0.063035  ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00935906  0.00673648]
   [ 0.01561124  0.02848843]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]]


 [[[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00330086  0.00659818]
   [ 0.00659818  0.063035  ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00935906  0.00673648]
   [ 0.01561124  0.02848843]]]


 [[[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.00330086  0.00659818]
   [ 0.

In [479]:
constant + linearFock + supposedLinearFluctuation + quadratic

array([[[[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00935906,  0.00673648],
         [ 0.01561124,  0.02848843]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [480]:
sum(residualTerms)

array([[[[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00935906,  0.00673648],
         [ 0.01561124,  0.02848843]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [481]:
sum(residualTerms[9:])

array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0

In [482]:
sum(residualTerms[:5]) + sum(residualTerms[9:])

array([[[[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00935906,  0.00673648],
         [ 0.01561124,  0.02848843]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [483]:
Ej = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [I,J], [A,B]), Vertex(t2Tensor, [A,B], [I,J])])])
Ek = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [I,J], [B,A]), Vertex(t2Tensor, [A,B], [I,J])])])

In [484]:
ECCD = 2 * contractTensorSum(Ej) - contractTensorSum(Ek)
print(ECCD)

0.0


In [485]:
def getCCDEnergy(doublesTensor):
    Ej = TensorSum([TensorProduct([h2Diagrams[3], doublesTensor], vertexList=[Vertex(h2Diagrams[3], [I,J], [A,B]), Vertex(doublesTensor, [A,B], [I,J])])])
    Ek = TensorSum([TensorProduct([h2Diagrams[3], doublesTensor], vertexList=[Vertex(h2Diagrams[3], [I,J], [B,A]), Vertex(doublesTensor, [A,B], [I,J])])])
    return 2 * contractTensorSum(Ej) - contractTensorSum(Ek)

In [486]:
t2Tensor.array * ECCD

array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0

In [487]:
print(Ej)
print(Ek)

1.0 * v_{IJ}^{AB}{t_{2}}_{AB}^{IJ}
1.0 * v_{IJ}^{BA}{t_{2}}_{AB}^{IJ}


Coupled cluster

In [488]:
print(t2Tensor.array.reshape((-1)))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [489]:
residualTensor = Tensor("R", ['p', 'p'], ['h', 'h'])

In [490]:
residualTensor.array = contractTensorSum(amplitudes)

In [491]:
print(residualTensor.array.reshape((-1)))

[ 0.00330086  0.00659818  0.00659818  0.063035    0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.00935906  0.00673648
  0.01561124  0.02848843  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.00330086  0.00659818  0.00659818  0.063035
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.00935906  0.00673648
  0.01561124  0.02848843  0.          0.          0.          0.
  0.          0.          0.          0.          0.00330086  0.00659818
  0.00659818  0.063035    0.          0.          0.          0.
  0.          0.          0.          0.          0.00935906  0.00673648
  0.01561124  0.02848843  0.          0.          0.          0.
  0.          0.          0.          0.          0.

In [492]:
print(t2Tensor.array)

[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0.

In [493]:
def doublesAmplitudesToVector(doublesAmplitudeTensor):
    vec = []
#    for i in range()
    return vec

In [494]:
def testAmplitudes(amplitudeVector):
    t2Tensor.array = amplitudeVector.reshape(t2Tensor.array.shape)
    residualTensor.array = contractTensorSum(amplitudes)
    return residualTensor.array.reshape((-1))

In [495]:
def getDoublesMP2Amplitudes(doublesTensor):
    mp2Amplitudes = np.zeros_like(doublesTensor.array)
    for i in range(mp2Amplitudes.shape[0]):
        for j in range(mp2Amplitudes.shape[1]):
            for k in range(mp2Amplitudes.shape[2]):
                for l in range(mp2Amplitudes.shape[3]):
#                    mp2Amplitudes[i,j,k,l] = -(2 - (i==j)) * h2Diagrams[12].array[i,j,k,l] / (fock[i + Nocc, i + Nocc] + fock[j + Nocc, j + Nocc] - fock[k, k] - fock[l, l])
                    mp2Amplitudes[i,j,k,l] = -h2Diagrams[12].array[i,j,k,l] / (fock[i + Nocc, i + Nocc] + fock[j + Nocc, j + Nocc] - fock[k, k] - fock[l, l])
    return mp2Amplitudes

In [496]:
def iterateDoublesAmplitudes(doublesTensor, residual):
    amplitudes = doublesTensor.array
    for i in range(amplitudes.shape[0]):
        for j in range(amplitudes.shape[1]):
            for k in range(amplitudes.shape[2]):
                for l in range(amplitudes.shape[3]):
#                    mp2Amplitudes[i,j,k,l] = (2 - (i==j)) * residual.array[i,j,k,l] / (fock[i + Nocc, i + Nocc] + fock[j + Nocc, j + Nocc] - fock[k, k] - fock[l, l])
                    amplitudes[i,j,k,l] -= residual.array[i,j,k,l] / (fock[i + Nocc, i + Nocc] + fock[j + Nocc, j + Nocc] - fock[k, k] - fock[l, l])
    return amplitudes

In [497]:
iterateDoublesAmplitudes(t2Tensor, h2Diagrams[12])

array([[[[-0.00034461, -0.00127549],
         [-0.00127549, -0.08213441]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[-0.00093956, -0.00121257],
         [-0.00281002, -0.02477347]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[-0.00034461, -0.00127549],
         [-0.00127549, -0.08213441]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [498]:
t2Tensor.array = np.zeros_like(t2Tensor.array)

In [499]:
Energy = getCCDEnergy(t2Tensor)

In [500]:
print(t2Tensor.array)

[[[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]

  [[0.

In [501]:
print(getDoublesMP2Amplitudes(t2Tensor))

[[[[-0.00034461 -0.00127549]
   [-0.00127549 -0.08213441]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.00093956 -0.00121257]
   [-0.00281002 -0.02477347]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]]


 [[[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.00034461 -0.00127549]
   [-0.00127549 -0.08213441]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.00093956 -0.00121257]
   [-0.00281002 -0.02477347]]]


 [[[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.         -0.        ]
   [-0.         -0.        ]]

  [[-0.00034461 -0.00127549]
   [-0.

In [502]:
def convergeDoublesAmplitudes():
    t2Tensor.array = np.zeros_like(t2Tensor.array)
#    t2Tensor.array = trueAmplitudeArray
    Energy = getCCDEnergy(t2Tensor)
    residualTensor.array = h2Diagrams[12].array
    while True:
        print(Energy)
        t2Tensor.array = iterateDoublesAmplitudes(t2Tensor, residualTensor)
        residualTensor.array = contractTensorSum(amplitudes)
        Energy = getCCDEnergy(t2Tensor)
#        newEnergy = getCCDEnergy(t2Tensor)
#        if abs(newEnergy - Energy) < 0.00000001:
        if np.all(abs(residualTensor.array) < 0.0000000000000001):
            break
#        Energy = newEnergy
    print(Energy)
    print(t2Tensor.array)

In [503]:
residualTensor.array

array([[[[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00935906,  0.00673648],
         [ 0.01561124,  0.02848843]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.00330086,  0.00659818],
         [ 0.00659818,  0.063035  ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[

In [504]:
trueCCD

(-0.04675404036274192,
 (array([[0.        , 0.        , 0.        , 0.00034378, 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.01691119, 0.        ,
          0.        , 0.        ]]),
  array([[0.        , 0.        , 0.        , 0.00034378, 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.01691119, 0.        ,
          0.        , 0.        ]])),
 (array([[[[ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,

In [505]:
convergeDoublesAmplitudes()

0.0
-0.024311010819762696
-0.035549815431841804
-0.04100158135429713
-0.0437073638104578
-0.045068230093333904
-0.0457570587227324
-0.04610686838249322
-0.04628483280767274
-0.04637547038416189
-0.046421665143156814
-0.04644522051463633
-0.046457235905851005
-0.04646336638372019
-0.046466494835433615
-0.04646809153055459
-0.04646890652857675
-0.046469322555975336
-0.046469534934192654
-0.04646964335551507
-0.04646969870731482
-0.04646972696638695
-0.04646974139387719
-0.046469748759825694
-0.046469752520539455
-0.046469754440598705
-0.04646975542090331
-0.04646975592140893
-0.046469756176948436
-0.046469756307417606
-0.04646975637403053
-0.046469756408040755
-0.046469756425405205
-0.046469756434270905
-0.04646975643879742
-0.04646975644110852
-0.04646975644228846
-0.04646975644289092
-0.0464697564431985
-0.04646975644335554
-0.04646975644343573
-0.04646975644347666
-0.04646975644349757
-0.046469756443508245
-0.04646975644351367
-0.04646975644351646
-0.046469756443517904
-0.046469756443

In [506]:
residualTensor.array

array([[[[-3.86416431e-18, -2.77149180e-18],
         [-2.34458720e-18,  2.77555756e-17]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-3.16769146e-18, -5.63446317e-18],
         [-5.90043151e-18,  7.15573434e-18]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]]],


       [[[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-7.78762092e-18, -3.60497222e-18],
         [-2.79182059e-18,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.000

In [507]:
trueCCD

(-0.04675404036274192,
 (array([[0.        , 0.        , 0.        , 0.00034378, 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.01691119, 0.        ,
          0.        , 0.        ]]),
  array([[0.        , 0.        , 0.        , 0.00034378, 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.01691119, 0.        ,
          0.        , 0.        ]])),
 (array([[[[ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,  0.        ,  0.        ],
           [ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        ,

In [508]:
t2Tensor.array

array([[[[ 7.25057996e-05, -1.22582488e-03],
         [-1.22582488e-03, -1.71775864e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-8.32521512e-04, -1.22019934e-03],
         [-2.35858417e-03, -5.57834516e-02]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]]],


       [[[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 7.25057996e-05, -1.22582488e-03],
         [-1.22582488e-03, -1.71775864e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.000

In [509]:
trueAmplitudeArray

array([[[[ 7.19737878e-05, -1.22968729e-03],
         [-1.22968729e-03, -1.71537433e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-8.29953157e-04, -1.24378783e-03],
         [-2.37889655e-03, -5.73075335e-02]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]]],


       [[[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 7.19737878e-05, -1.22968729e-03],
         [-1.22968729e-03, -1.71537433e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.000

In [510]:
t2Tensor.array = getDoublesMP2Amplitudes(t2Tensor)
fsolve(testAmplitudes, t2Tensor.array.reshape((-1)))

array([-1.65613545e-04,  4.01613449e-04, -1.60993260e-03, -8.42624893e-02,
       -2.10554002e-05,  9.62412955e-04, -1.72545736e-05, -1.39926005e-04,
       -3.17559948e-04,  5.04645089e-04, -3.02962237e-05,  2.59254715e-04,
       -1.67782552e-02, -4.19112947e-02,  1.75543490e-05, -4.65048544e-03,
       -1.77944386e-03,  1.70441566e-03, -3.32830111e-03,  1.38745241e-02,
        2.67643422e-05, -4.34123224e-04, -8.48244256e-05,  7.96208373e-05,
        5.70886148e-05, -1.07539504e-03, -4.02356578e-04, -1.02058967e-03,
       -1.38487751e-04, -6.83968251e-04, -2.42040449e-05, -4.48340828e-04,
       -7.78151052e-05,  6.54136187e-04, -1.34817020e-03, -1.75084494e-01,
        4.97249556e-05, -4.10386392e-04,  1.19928346e-03,  5.34081553e-03,
       -4.92072985e-03, -4.06903963e-02, -2.57584458e-02,  2.15582527e-02,
       -2.05043811e-06, -1.40047660e-03, -4.67215774e-05, -1.07089447e-04,
        3.57594944e-05,  8.10874457e-06,  2.98595364e-04,  1.30884119e-03,
       -1.38307292e-03,  

In [511]:
t2Tensor.array

array([[[[6.90673436e-310, 6.90673436e-310],
         [6.90673436e-310, 6.90673436e-310]],

        [[6.90673436e-310, 6.90673436e-310],
         [8.06315134e-321, 8.06315134e-321]],

        [[0.00000000e+000, 6.90673480e-310],
         [            nan, 0.00000000e+000]],

        [[6.90682573e-310, 9.68368666e-322],
         [6.90673436e-310, 6.90673480e-310]],

        [[6.90673436e-310, 6.90673436e-310],
         [6.90673436e-310, 6.90673436e-310]],

        [[6.90673436e-310, 6.90673436e-310],
         [6.90673436e-310, 6.90673436e-310]],

        [[6.90673436e-310, 6.90673436e-310],
         [6.90673436e-310, 6.90673436e-310]]],


       [[[6.90673436e-310, 6.90673436e-310],
         [6.90673436e-310, 6.90673436e-310]],

        [[6.90673436e-310, 6.90673480e-310],
         [6.90673480e-310, 6.90673481e-310]],

        [[6.90675859e-310, 6.90675859e-310],
         [6.90675859e-310, 6.90673478e-310]],

        [[6.90673481e-310, 6.90673478e-310],
         [6.90673478e-310, 6.9067

In [512]:
2 * contractTensorSum(Ej) - contractTensorSum(Ek)

7.951473638268446e+177

In [513]:
trueAmplitudeArray

array([[[[ 7.19737878e-05, -1.22968729e-03],
         [-1.22968729e-03, -1.71537433e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-8.29953157e-04, -1.24378783e-03],
         [-2.37889655e-03, -5.73075335e-02]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]]],


       [[[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 7.19737878e-05, -1.22968729e-03],
         [-1.22968729e-03, -1.71537433e-01]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.000

In [514]:
residualTensor.array

array([[[[ 2.06167819e-18,  1.27122705e-17],
         [ 1.21701694e-17,  6.17995238e-17]],

        [[ 1.46096243e-17, -1.24242793e-17],
         [-1.72117095e-17,  6.86774314e-18]],

        [[ 3.77607288e-18, -7.30057697e-18],
         [-4.33680869e-18,  1.49204854e-18]],

        [[-2.92226367e-17,  1.88333541e-17],
         [ 1.69813165e-17,  4.55364912e-18]],

        [[ 2.38668474e-17, -1.74759838e-17],
         [ 8.40256684e-19, -1.45283091e-16]],

        [[-7.87232421e-18,  5.41338757e-18],
         [ 1.24005623e-18, -2.65968345e-18]],

        [[-7.96888597e-18,  1.63985579e-17],
         [-2.49366500e-18, -6.08508469e-18]]],


       [[[ 1.43025748e-17, -1.71381235e-17],
         [-1.26716129e-17,  6.94905830e-18]],

        [[-1.63477359e-17,  2.48959924e-17],
         [ 2.58582218e-17,  2.25514052e-16]],

        [[-4.09286320e-18, -2.54353406e-18],
         [-1.24683250e-18, -4.12674452e-18]],

        [[-2.75793928e-17,  1.34796823e-17],
         [ 1.73472348e-17, -2.046

In [515]:
def amplitudesToVector(amplitudeTensor):
    amplitudeArray = amplitudeTensor.array

In [516]:
from importlib import reload

In [517]:
import texify
#reload(texify)

In [518]:

texify.texify([Ej, Ek], "CCDEnergyTerms")

In [519]:
t2Tensor.array = trueAmplitudeArray

In [520]:
2 * contractTensorSum(Ej) - contractTensorSum(Ek)

-0.046767626212592964

In [521]:
getDoublesMP2Amplitudes(t2Tensor)

array([[[[-0.00034461, -0.00127549],
         [-0.00127549, -0.08213441]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.00093956, -0.00121257],
         [-0.00281002, -0.02477347]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]]],


       [[[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.00034461, -0.00127549],
         [-0.00127549, -0.08213441]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[-0.        , -0.        ],
         [-0.        , -0.        ]],

        [[

In [522]:
t2Tensor.array = getDoublesMP2Amplitudes(t2Tensor)
2 * contractTensorSum(Ej) - contractTensorSum(Ek)

-0.0243110108197627

In [523]:
from pyscf import mp

In [524]:
trueMP2 = mp.MP2(mf)

In [525]:
trueMP2.kernel()

(-0.024311010805774455,
 (array([[[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             2.16840434e-19],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  2.16840434e-19,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00,  0.00000000e+00, -2.16840434e-19,
             0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
             0.00000000e+00],
           [ 0.00000000e+00, -2.16840434